In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import torch
from datasets import Dataset

In [2]:
cache_dir = '/mnt/sdc/Debiasing/pretrained/bart'

In [3]:
model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli", cache_dir=cache_dir)
device = torch.device(f'cuda:6' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli", cache_dir=cache_dir)

In [4]:
classifier = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer, device=device)

In [15]:
sequence_to_classify = "A photo of young person"
candidate_labels = ['boy'] * 1000
res = classifier(sequence_to_classify, candidate_labels, multi_label=True)
print(res['sequence'])
print(res['labels'])
print(res['scores'])

--- Logging error ---
Traceback (most recent call last):
  File "/mnt/sdc/zungwooker/anaconda3/envs/gene/lib/python3.9/logging/__init__.py", line 1079, in emit
    msg = self.format(record)
  File "/mnt/sdc/zungwooker/anaconda3/envs/gene/lib/python3.9/logging/__init__.py", line 923, in format
    return fmt.format(record)
  File "/mnt/sdc/zungwooker/anaconda3/envs/gene/lib/python3.9/logging/__init__.py", line 659, in format
    record.message = record.getMessage()
  File "/mnt/sdc/zungwooker/anaconda3/envs/gene/lib/python3.9/logging/__init__.py", line 363, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/mnt/sdc/zungwooker/anaconda3/envs/gene/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/mnt/sdc/zungwooker/anaconda3/envs/gene/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/mnt/sdc/zungwooker/anaconda3/envs/ge

A photo of young person
['boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'boy', 'b

In [6]:
filtered_scores_labels = [(score, label) for score, label in zip(res['scores'], res['labels']) if score <= 0.99]
res['scores'], res['labels'] = zip(*filtered_scores_labels) if filtered_scores_labels else ([], [])

In [7]:
print(res['sequence'])
print(res['labels'])
print(res['scores'])

A photo of young
('girl', 'woman', 'person', 'face', 'pose', 'blue', 'love', 'hair', 'dress', 'mouse', 'laptop', 'costume', 'code', 'computer')
(0.9812822341918945, 0.8993655443191528, 0.8792117238044739, 0.6589139699935913, 0.4070006310939789, 0.17151901125907898, 0.11285580694675446, 0.03831982612609863, 0.02029099501669407, 0.006456996314227581, 0.0019525282550603151, 0.0011518665123730898, 0.0011386561673134565, 0.0004380836326163262)
